In [1]:
# Name Dipika Sharma
# DSC 540 T302 Data Preparation
# Week 7 and 8

I am using the https://www.geonames.org/postal-codes/US/CA/california.html site to get all the postal codes of California states.

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

In [79]:
# Declaring all the libraries I want to use now.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Download the following HTML file, save it and then use bs4 to read it from the disk
fd = open("Postal Codes California, United States.html", "r")
soup = BeautifulSoup(fd)
fd.close()
# print(soup.prettify())

In [5]:
# findall method to extract the content

Tables_ = soup.find_all("table")
print("There are {} tables in a source".format(len(Tables_)))

There are 3 tables in a source


In [84]:
# Let see which class we want to use for our analysis.
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
None
None
['restable']


As we can see we will be using the class 'restable'

In [7]:
# going to the website, right click, choose inspect. You can find the class of the table which we can use to find the correct table.

tbl = soup.find("table", {"class": "restable"})
print(type(tbl))
# soup.find("table", {"class": "restable"})

<class 'bs4.element.Tag'>


In [14]:
# Extracting all the headers from the class restable.

Header_1 = [th.getText().strip() for th in soup.find("table", {"class": "restable"}).findAll('th')]

In [15]:
Header_1

['', 'Place', 'Code', 'Country', 'Admin1', 'Admin2', 'Admin3']

Above is all the headers from class restable

In [25]:
# Extracting all data from class restable

data_ = [[td.get_text().strip() for td in tr.findAll('td')] for tr in soup.find("table", {"class": "restable"}).findAll('tr')]

In [31]:
# creating dataframe using rows and headers extract from source first.

postal_code = pd.DataFrame(data_, columns=Header_1)

In [32]:
# Lets see the top 5 rows of our extracted dataset.

postal_code.head()

,,Place,Code,Country,Admin1,Admin2,Admin3
0,None,None,None,None,None,None,None
1,1,Beverly Hills,90210,United States,California,Los Angeles,
2,,34.09/-118.406,None,None,None,None,None
3,2,Los Angeles,90002,United States,California,Los Angeles,
4,,33.95/-118.246,None,None,None,None,None


In [33]:
# Shape will show the number of rows and columns in our dataset.

postal_code.shape

(402, 7)

# Step 1 - Removing bad data

As we can see above postal_code dataframe has so many empty rows and also it is showing the laitude and longtitude of that place in dataset that we do not need in our analysis.
Lets remove those empty rows and place map map location from dataset.

In [65]:
# dropna function to drop nulls from our dataset.

postal_code_df = postal_code.dropna()

In [66]:
# Lets check the existing rows and columns of dataset now after dropping null values.

postal_code_df.shape

(200, 7)

# Step 2 - Changing the headers to make more sense

In [67]:
# Almost every column in this dataset has name which do not give the insight of the data.
# Lets rename columns so that it will be easy for us to read the data.

postal_code_df.rename(columns = {'':'ID', 'Place':'Place', 'Code':'Code', 'Country':'Country', 'Admin1':'State', 'Admin2':'City'}, inplace = True)

In [68]:
# lets check the data now with new columns.

postal_code_df.head()

,ID,Place,Code,Country,State,City,Admin3
1,1,Beverly Hills,90210,United States,California,Los Angeles,
3,2,Los Angeles,90002,United States,California,Los Angeles,
5,3,Los Angeles,90003,United States,California,Los Angeles,
7,4,Los Angeles,90004,United States,California,Los Angeles,
9,5,Los Angeles,90006,United States,California,Los Angeles,


# Step 3 - Droping columns

As we can see ID and Admin3 columns do not give any information about the data and we will not be using this column in our analysis. So it will be save to drop this column from our dataset.

In [69]:
postal_code_df = postal_code_df.drop(['ID','Admin3'], axis=1)

In [70]:
# lets check the first 5 rows to see how data looks now.

postal_code_df.head()

,Place,Code,Country,State,City
1,Beverly Hills,90210,United States,California,Los Angeles
3,Los Angeles,90002,United States,California,Los Angeles
5,Los Angeles,90003,United States,California,Los Angeles
7,Los Angeles,90004,United States,California,Los Angeles
9,Los Angeles,90006,United States,California,Los Angeles


# Step 4 - Looking for NULL values in dataframe

In [71]:
# checking the dataframe for null values, below function will count the number of NULL values for each column.

postal_code_df.isnull().sum()

Place      0
Code       0
Country    0
State      0
City       0
dtype: int64

As we can see our dataframe is all clean and do not have any null values. We are good to go.

# Step 5 - Removing the leading and ending whitespace

It is always good to check and get rid off the leadinf and ending space from values. As this can impact our analysis.

In [72]:
# using strip fucntion to remove leading and ending whitespace.

def strip_whitespace(s):
    return s.strip()  # using strip fucntion to remove leading and ending whitespace.

In [74]:
# Lets strip the whitespace if any in Place column of the dataset

postal_code_df['Place_stripped']=postal_code_df['Place'].apply(strip_whitespace)
postal_code_df['Place']=postal_code_df['Place_stripped']
postal_code_df.drop(labels=['Place_stripped'],axis=1,inplace=True)

# Step 6 - Check for duplicates

In [76]:
# Lets make sure we do not have any duplicates for columns Place and Code in our dataset

print("Is column Place is duplicated? - {}".format(any(postal_code_df.Place.duplicated())))
print("Is column Code is duplicated? - {}".format(any(postal_code_df.Code.duplicated())))

Is column Place is duplicated? - True
Is column Code is duplicated? - False


It make sense to have duplicates values for Place as we can have same place name for different postal code.
But we do not have same postal code for different place. So it is good to know that we have all unique code in our dataset for California State.

# Step 7 - Check for NaN values

In [77]:
# Lets check few columns of the dataset for NaN Values

print("Is the column Place contains NaN values? - %r " % postal_code_df.Place.isnull().values.any())
print("Is the column Code contains NaN values? - %r " % postal_code_df.Code.isnull().values.any())
print("Is the column City contains NaN values? - %r " % postal_code_df.City.isnull().values.any())

Is the column Place contains NaN values? - False 
Is the column Code contains NaN values? - False 
Is the column City contains NaN values? - False 


All the above columns showing 'false' for NaN values,so we are good to go and we can move ahead and start working on this dataset now.

In [83]:
postal_code_df

,Place,Code,Country,State,City
1,Beverly Hills,90210,United States,California,Los Angeles
3,Los Angeles,90002,United States,California,Los Angeles
5,Los Angeles,90003,United States,California,Los Angeles
7,Los Angeles,90004,United States,California,Los Angeles
9,Los Angeles,90006,United States,California,Los Angeles
...,...,...,...,...,...
391,Inglewood,90310,United States,California,Los Angeles
393,Inglewood,90311,United States,California,Los Angeles
395,Inglewood,90312,United States,California,Los Angeles
397,Santa Monica,90402,United States,California,Los Angeles


This dataset is simple and goal of using this dataset is to get all the postal codes of California states for my project work in which I will analysis the covid and vaccination data for California states.